In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../data/KRI-DAC_Jeju_data5.txt")

FileNotFoundError: [Errno 2] File b'../data/KRI-DAC_Jeju_data5.txt' does not exist: b'../data/KRI-DAC_Jeju_data5.txt'

In [ ]:
df.head()

### 좌표 변환

참조: https://m.blog.naver.com/wideeyed/221243506770

In [ ]:
from pyproj import Proj, transform

In [ ]:
proj_UTMK = Proj(init='epsg:5178')
proj_WGS84 = Proj(init='epsg:4326')
x1, y1 = 877005.9834, 1.479766e+06
x2, y2 = transform(proj_UTMK, proj_WGS84, x1, y1)

In [ ]:
x2, y2

In [ ]:
%%time
def coor_projector(x, y):
    return transform(proj_UTMK, proj_WGS84, x, y)
coor = df.apply(lambda row: coor_projector(row.POINT_X, row.POINT_Y), axis=1)

In [ ]:
df = df.assign(
    lon=coor.apply(lambda x: x[0]),
    lat=coor.apply(lambda x: x[1])
)

### plotly 시각화

참조1: http://blog.daum.net/geoscience/1420
참조2: https://plotly.com/python/scattermapbox/

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
sample_df = df.head(1000)

In [ ]:
fig = px.scatter_mapbox(
    sample_df, 
    lat='lat', lon='lon', 
    hover_name='OBJECTID', hover_data= ['TotalSpent'],
    color_discrete_sequence=["fuchsia"], zoom=10, height=400
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(mapbox=dict(
    center=go.layout.mapbox.Center(
        lat=sample_df.lat.median(),
        lon=sample_df.lon.median()
    )
))
fig.show()